# Time-independent analysis

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

Load libraries

In [3]:
from fastai.imports import *
from fastai.structured import *
from sklearn.model_selection import train_test_split, KFold
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn import metrics
from dill import load_session

Load environment

In [4]:
load_session('tmp/postprocessing.pkl')

__Target Variable: normalized volume (time independent analysis)__

We decide at first to use `norm_volume_all_days` as target variable. We remove all the other variables coming from the `sales` dataset.
We do not use types as feature at the beginning.

In [5]:
flds = ['store_code','norm_volume_selling_days']
final_y = df_shops_days_filer[flds]

NameError: name 'df_shops_days_filer' is not defined

In [6]:
final_no_time = final_y.merge(metadata_no_day, how='left', on='store_code')
# Remove types at beginning, maybe added later in a second step
final_no_time = final_no_time[final_no_time.columns.drop(list(final_no_time.filter(regex='types')))]

def coherce_type(df, reg, typ):
    float_columns = [x for x in df.columns if reg in x]
    for cl in float_columns:
        df[cl] = df[cl].astype(typ)
    return df

## Float columns
final_no_time = coherce_type(final_no_time, '_mean_', 'float64')

## Integer column
final_no_time = coherce_type(final_no_time, '_n', 'int')

NameError: name 'final_y' is not defined

Remove columns with missing values.

In [7]:
complete_null_columns = final_no_time.isnull().all()
col_to_remove = complete_null_columns[complete_null_columns == True]
col_to_remove

NameError: name 'final_no_time' is not defined

We remove the column `movie_rental_mean_rating` due to missing values.

__Preparation dataset: using normalized volume as target variable and surroundings types (how many, rating,...) as features__

In [8]:
final_no_time = final_no_time[final_no_time.columns.drop('movie_rental_mean_rating')]

NameError: name 'final_no_time' is not defined

__Division between training/validation/test sets: 70/20/10.__

In [ ]:
train, test = train_test_split(final_no_time[final_no_time.columns.drop('store_code')], test_size=0.10)
train.shape[0], test.shape[0]

Training set sample size: 431 POSs, 48 POSs in the test set. 

We select also the additional validation set (20%).

In [ ]:
def split_vals(a,n): 
    return a[:n].copy(), a[n:].copy()

n_valid = 60  # same as Kaggle's test set size
n_trn = len(X)-n_valid
X_train, X_valid = split_vals(X, n_trn)
y_train, y_valid = split_vals(y, n_trn)

X_train.shape, y_train.shape, X_valid.shape

## Linear regression

In [ ]:
# for linear regression, scaling is important
df, y, nas, mapper = proc_df(train, 'norm_volume_selling_days', do_scale=True)
X_train, X_valid = split_vals(df, n_trn)

In [ ]:
m = LinearRegression().fit(X_train, y_train)
m.score(X_valid, y_valid)

In [ ]:
m.score(X_train, y_train)

In [ ]:
preds = m.predict(X_valid)
rmse(preds, y_valid)

In [ ]:
plt.scatter(preds, y_valid, alpha=1, s=2);

## Random forest regression

As control, if any variable is categorical, we provide the numeric coding required for a random forest. Therefore we call `train_cats` to convert strings to pandas categories.

In [ ]:
train_cats(train)

We'll replace categories with their numeric codes, handle missing continuous values, and split the dependent variable into a separate variable.

In [ ]:
X, y, nas = proc_df(df = train, y_fld='norm_volume_selling_days')

After splitting training and test sets, we might have columns with complete missing values. We set them to 0. They will not be informative.

In [ ]:
X.fillna(0, inplace=True)

In [11]:
def rmse(x,y):
    return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [ ]:
m = RandomForestRegressor(n_jobs=-1)
m.fit(X, y)
print_score(m)

### Bagging

In [ ]:
preds = np.stack([t.predict(X_valid) for t in m.estimators_])
preds[:,0], np.mean(preds[:,0]), y_valid[0]

In [ ]:
plt.plot([metrics.r2_score(y_valid, np.mean(preds[:i+1], axis=0)) for i in range(15)]);

We are clearly performing very badly!!

Need to rethink the target variable in case of time-independent analysis.